# GFF Processing and Genomic Feature Annotation

This notebook details the process of parsing a GFF3 file to extract and annotate genomic features. The primary focus is on identifying coding genes, their constituent parts (CDS, introns), determining primary transcripts, and then finding and annotating intergenic regions. This workflow is crucial for many downstream genomic analyses, such as understanding gene regulation or mapping sequencing reads to specific genomic contexts.

## 1. Setup and Load Modules

We begin by importing the necessary Python libraries:
- `numpy`: For numerical operations, particularly for handling `NaN` (Not a Number) values.
- `pandas`: For data manipulation and analysis, providing DataFrame structures to handle tabular data like GFF and BED files.
- `re`: The regular expression module, essential for parsing complex attribute strings found in GFF files.
- `pybedtools`: A library for genomic interval manipulation, used here for operations like finding complementary regions (intergenic regions).
- `snakemake`: A workflow management system for reproducible computational pipelines.

In [103]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import pybedtools
from pybedtools import BedTool
# from snakemake.script import snakemake


In [104]:
def update_sysID(genes, gene_IDs_names_products):
    synonyms2ID = gene_IDs_names_products.query("gene_type == 'protein coding gene'").set_index("gene_systematic_id")["synonyms"].str.split(",").explode().str.strip().dropna().reset_index().set_index("synonyms")
    names2ID = gene_IDs_names_products.query("gene_type == 'protein coding gene'").set_index("gene_name")["gene_systematic_id"].drop_duplicates().reset_index().set_index("gene_name")
    sysIDs_now = gene_IDs_names_products.query("gene_type == 'protein coding gene'")["gene_systematic_id"].unique().tolist()
    updated_sysIDs = []
    for gene in genes:
        if pd.isna(gene):
            updated_sysIDs.append(gene)
            print(f"{gene} is NA")
        elif gene in sysIDs_now:
            updated_sysIDs.append(gene)
        elif gene in names2ID.index:
            updated = names2ID.loc[gene, "gene_systematic_id"]
            if isinstance(updated, str):
                updated_sysIDs.append(updated)
                print(f"{gene} is updated to {updated}")
            else:
                updated_sysIDs.append(np.nan)
                print(f"{gene} has multiple updates:", updated)
        elif gene in synonyms2ID.index:
            updated = synonyms2ID.loc[gene, "gene_systematic_id"]
            if isinstance(updated, str):
                updated_sysIDs.append(updated)
                print(f"{gene} is updated to {updated}")
            else:
                updated_sysIDs.append(np.nan)
                print(f"{gene} has multiple updates:", updated)
        else:
            updated_sysIDs.append(gene)
            print(f"{gene} is not found in geneid2symbol or synonyms2ID")
    return updated_sysIDs

## 2. Configuration: File Paths

### 2.1 Define the paths to the input files.

Centralizing these paths makes it easy to update file locations if the data moves or if different versions of files are used.

- `gff_file_path`: Path to the GFF3 file containing genomic annotations.
- `fai_file_path`: Path to the FASTA index file (`.fai`). This file provides chromosome names and their sizes, which is required by `pybedtools` for operations like `complement`.
- `peptide_stats_path`: Path to a TSV (Tab Separated Values) file containing statistics about peptides (e.g., protein length). This is used to help identify primary protein-coding transcripts.

In [105]:
gff_file_path = "/data/c/yangyusheng_optimized/DIT_HAP_pipeline/resources/pombase_data/2025-05-01/gff/Schizosaccharomyces_pombe_all_chromosomes.gff3"
fai_file_path = "/data/c/yangyusheng_optimized/DIT_HAP_pipeline/resources/pombase_data/2025-05-01/fasta/Schizosaccharomyces_pombe_all_chromosomes.fa.fai"
peptide_stats_path = "/data/c/yangyusheng_optimized/DIT_HAP_pipeline/resources/pombase_data/2025-05-01/Protein_features/PeptideStats.tsv"
gene_IDs_names_products_path = "/data/c/yangyusheng_optimized/DIT_HAP_pipeline/resources/pombase_data/2025-05-01/Gene_metadata/gene_IDs_names_products.tsv"
FYPOviability_path = "/data/c/yangyusheng_optimized/DIT_HAP_pipeline/resources/pombase_data/2025-05-01/Gene_metadata/FYPOviability.tsv"
Hayles_viability_path = "/data/c/yangyusheng_optimized/DIT_HAP_pipeline/resources/Hayles_2013_OB_merged_categories.xlsx"

In [106]:
# # snakemake input arguments
# gff_file_path = snakemake.input[0]
# fai_file_path = snakemake.input[1]
# peptide_stats_path = snakemake.input[2]
# gene_IDs_names_products_path = snakemake.input[3]
# FYPOviability_path = snakemake.input[4]
# Hayles_viability_path = snakemake.params[0]

### 2.2 Define the paths to the output files.

- `primary_transcripts_bed_path`: Path to save the primary transcripts in BED format.
- `intergenic_regions_bed_path`: Path to save the intergenic regions in BED format.
- `non_coding_rna_bed_path`: Path to save the non-coding RNAs in BED format.
- `Genome_intervals_bed_path`: Path to save the genome intervals in BED format.


In [128]:
output_dir = Path("/data/c/yangyusheng_optimized/DIT_HAP_pipeline/resources/pombase_data/2025-05-01/genome_region")
output_dir.mkdir(parents=True, exist_ok=True)
coding_gene_primary_transcripts_bed_path = output_dir / "coding_gene_primary_transcripts.bed"
intergenic_regions_bed_path = output_dir / "intergenic_regions.bed"
non_coding_rna_bed_path = output_dir / "non_coding_rna.bed"
Genome_intervals_bed_path = output_dir / "Genome_intervals.bed"
overlapped_region_bed_path = output_dir / "overlapped_region.bed"

In [108]:
# # snakemake output arguments
# primary_transcripts_bed_path = snakemake.output[0]
# intergenic_regions_bed_path = snakemake.output[1]
# non_coding_rna_bed_path = snakemake.output[2]
# Genome_intervals_bed_path = snakemake.output[3]
# overlapped_region_bed_path = snakemake.output[4]

## 3. GFF File Parsing
The GFF3 format is complex. The 9th column, "Attribute", contains a semicolon-separated list of tag-value pairs. These functions help extract meaningful information like Gene IDs and Transcript IDs.

### 3.1. Extracting Transcript Identifiers from GFF Attributes

The `get_gff_transcript_id` function is a specialized utility for parsing transcript identifiers from the 'Attribute' column of a GFF DataFrame. This is crucial for linking features like exons and CDS back to their parent transcripts, as well as identifying the primary IDs for transcript features themselves.

The function operates row-wise and applies the following logic:
1.  **Primary Transcript Features**: If the feature type (e.g., `mRNA`, `tRNA`, `lncRNA`, as defined in `transcript_feature_types`) is a primary transcript, the function parses its `ID` attribute. The identifier is extracted from the `ID=` field, taking the value up to the first colon (`:`) or semicolon (`;`). This design allows for handling IDs that might have sub-parts (e.g., `ID=transcript_XYZ:part1` would yield `transcript_XYZ`).
2.  **Sub-Features**: For features that are components of a transcript (e.g., `CDS`, `exon`, `intron`), the function looks for a `Parent=` attribute. The value associated with `Parent` is extracted (up to the first semicolon) and considered the transcript identifier.
3.  **Fallback**: If an identifier cannot be determined using these rules (e.g., the attribute string is malformed or lacks the expected fields), the function returns `np.nan`.

This targeted approach ensures consistent and accurate retrieval of transcript identifiers, which are essential for downstream analysis and associating various genomic features with their respective transcripts.

In [109]:
def get_gff_transcript_id( # Function renamed for clarity
    row,
    # Default arguments for compiled regex patterns and feature types.
    # These are compiled/created only once when the function is defined, improving performance.
    id_pattern=re.compile(r"ID=([^:;]+)"),
    parent_pattern=re.compile(r"Parent=([^;]+)"),
    transcript_feature_types=frozenset(["mRNA", "tRNA", "rRNA", "snoRNA", "snRNA", "lncRNA"])
):
    """
    Extracts a transcript identifier from a GFF row's attributes.

    This function is designed to be applied row-wise to a pandas DataFrame
    derived from a GFF file.

    Rules for extraction:
    1. If the feature type (from `row["Feature"]`) is in `transcript_feature_types`
       (e.g., "mRNA", "lncRNA"), the function attempts to parse its own 'ID' attribute.
       The ID is taken as the part of the string after "ID=" and before any
       colon (':') or semicolon (';'). This handles cases like "ID=transcript_ABC;..."
       or "ID=transcript_XYZ:part1;...".
    2. If the feature type is not a primary transcript type but the attribute string
       (from `row["Attribute"]`) contains "Parent=", the function attempts to parse
       the 'Parent' ID. This typically applies to sub-features like "CDS", "exon",
       or "intron", where their 'Parent' attribute links to the transcript ID.
       The Parent ID is taken as the part of the string after "Parent=" and
       before any semicolon (';').
    3. If neither of these conditions yields an identifier, `np.nan` is returned.

    Args:
        row (pd.Series): A row from a GFF DataFrame. Must contain 'Feature'
                         and 'Attribute' columns.
        id_pattern (re.Pattern, optional): Compiled regular expression for parsing
                                           IDs of primary transcript features.
                                           Defaults to a pre-compiled pattern.
        parent_pattern (re.Pattern, optional): Compiled regular expression for
                                               parsing Parent IDs of sub-features.
                                               Defaults to a pre-compiled pattern.
        transcript_feature_types (frozenset, optional): An immutable set of feature
                                                        type strings that are
                                                        considered primary transcripts.
                                                        Defaults to a pre-defined set.

    Returns:
        str or np.nan: The extracted transcript identifier, or np.nan if not found
                       or if the attribute string is not as expected.
    """
    # Ensure 'Attribute' is a string to handle potential non-string values (e.g., NaN) gracefully.
    attributes = str(row["Attribute"])
    feature_type = row["Feature"]

    if feature_type in transcript_feature_types:
        match = id_pattern.search(attributes)
        if match:
            return match.group(1)
    # For sub-features (e.g., CDS, exon), their 'Parent' ID is the transcript ID.
    # Checking for "Parent=" substring first can be a small optimization.
    elif "Parent=" in attributes: 
        match = parent_pattern.search(attributes)
        if match:
            return match.group(1)
    
    return np.nan

### 3.2. Main GFF Parsing Function

The `parse_gff_data` function reads the GFF3 file into a pandas DataFrame and performs initial processing:
1. Reads the GFF file, skipping comments and assigning standard column names.
2. Extracts `Systematic ID` and `Transcript` from the attribute column.
   - For `gene` features, `Systematic ID` comes from its `ID` attribute.
   - For transcript features (mRNA, tRNA, etc.), `Systematic ID` comes from their `Parent` attribute, and `Transcript` from their `ID` attribute.
   - For sub-transcript features (CDS, exon, intron), `Transcript` comes from their `Parent` attribute. Their `Systematic ID` can be filled by merging/mapping based on `Transcript` later if necessary, or by ensuring transcript features carry their parent gene's ID.
3. Defines a specific order for chromosomes using `pd.Categorical`. This ensures consistent sorting, vital for genomic analyses.
4. Drops rows where essential identifiers like `Systematic ID` or `Transcript` could not be parsed (relevant for transcript-focused parts).

In [110]:
def parse_gff_data(gff_file_path):
    """Reads and parses a GFF3 file, extracting Systematic ID and Transcript.

    Args:
        gff_file_path (str): Path to the GFF3 file.
        
    Returns:
        pd.DataFrame: A DataFrame containing the parsed GFF data with 'Systematic ID' and 'Transcript' columns.
    """
    column_names = [ 
        "Chr", "Source", "Feature", "Start", "End", 
        "Score", "Strand", "Frame", "Attribute" 
    ]
    gff_df = pd.read_csv(
        gff_file_path,
        sep="\t",
        comment="#",
        names=column_names,
        dtype={"Chr": str, "Start": pd.Int64Dtype(), "End": pd.Int64Dtype()} 
    )

    # Extract Systematic ID
    # For gene features, Systematic ID is from 'ID'. For transcript features, Systematic ID is from 'Parent'.
    extract_systematic_ID_pattern = re.compile(r"ID=(\S+?)(?:$|(?:|\.\d(?::\S+|));)")
    gff_df["Systematic ID"] = gff_df["Attribute"].str.extract(
        extract_systematic_ID_pattern, expand=False
    )

    # Extract Transcript
    # For transcript features, Transcript is from 'ID'. For sub-transcript features, Transcript is from 'Parent'.
    gff_df["Transcript"] = gff_df.apply(get_gff_transcript_id, axis=1)


    # Define chromosome order for consistent sorting
    chr_order = ["chr_II_telomeric_gap", "I", "II", "III", "mating_type_region", "mitochondrial"]
    gff_df["Chr"] = pd.Categorical(gff_df["Chr"], categories=chr_order, ordered=True)
    
    return gff_df

### 3.3. Load and Inspect GFF Data

We now use the `parse_gff_data` function to load and parse the GFF file. We then display the head, info, and some summary statistics of the resulting DataFrame to ensure it has been loaded and processed correctly.

In [111]:
gff_df = parse_gff_data(gff_file_path)

print("GFF Data Head:")
display(gff_df.head())
print("\nGFF Data Info:")
gff_df.info()
print(f"\nShape of GFF DataFrame: {gff_df.shape}")
print(f"Number of unique Systematic IDs: {gff_df['Systematic ID'].nunique()}\nNumber of unique Transcripts: {gff_df['Transcript'].nunique()}")

print("\nUnique feature types in GFF (post-parse):")
display(gff_df["Feature"].value_counts())

GFF Data Head:


,Chr,Source,Feature,Start,End,Score,Strand,Frame,Attribute,Systematic ID,Transcript
0,I,PomBase,gene,1798347,1798835,.,+,.,ID=SPAC1002.01;Name=mrx11;characterisation_sta...,SPAC1002.01,NaN
1,I,PomBase,mRNA,1798347,1798835,.,+,.,ID=SPAC1002.01.1;Parent=SPAC1002.01,SPAC1002.01,SPAC1002.01.1
2,I,PomBase,CDS,1798347,1798835,.,+,0,ID=SPAC1002.01.1:exon:1;Parent=SPAC1002.01.1,SPAC1002.01,SPAC1002.01.1
3,I,PomBase,gene,1799014,1800053,.,+,.,ID=SPAC1002.02;Name=pom34;characterisation_sta...,SPAC1002.02,NaN
4,I,PomBase,mRNA,1799014,1800053,.,+,.,ID=SPAC1002.02.1;Parent=SPAC1002.02,SPAC1002.02,SPAC1002.02.1



GFF Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58568 entries, 0 to 58567
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Chr            58568 non-null  category
 1   Source         58568 non-null  object  
 2   Feature        58568 non-null  object  
 3   Start          58568 non-null  Int64   
 4   End            58568 non-null  Int64   
 5   Score          58568 non-null  object  
 6   Strand         58568 non-null  object  
 7   Frame          58568 non-null  object  
 8   Attribute      58568 non-null  object  
 9   Systematic ID  58568 non-null  object  
 10  Transcript     45363 non-null  object  
dtypes: Int64(2), category(1), object(8)
memory usage: 4.6+ MB

Shape of GFF DataFrame: (58568, 11)
Number of unique Systematic IDs: 13205
Number of unique Transcripts: 12767

Unique feature types in GFF (post-parse):


Feature
gene                                       12685
CDS                                        10321
exon                                        7574
lncRNA                                      7166
intron                                      5337
mRNA                                        5145
three_prime_UTR                             4798
five_prime_UTR                              4635
long_terminal_repeat                         238
tRNA                                         196
snoRNA                                        66
promoter                                      62
low_complexity_region                         59
rRNA                                          49
repeat_region                                 44
sncRNA                                        38
pseudogenic_transcript                        31
region                                        28
nuclear_mt_pseudogene                         17
dh_repeat                                     16
LTR_retrotra

## 4. Load Peptide Statistics

Peptide statistics, such as protein length, are valuable for validating transcript models, especially in identifying the primary protein-coding transcript among potential alternatives for a gene. We load this data from the `PeptideStats.tsv` file and create a dictionary mapping `Systematic ID` to protein length (number of residues) for quick lookup.

In [112]:
peptide_stats_df = pd.read_csv(peptide_stats_path, sep="\t")
# Create a dictionary mapping Systematic ID (Systematic_ID in the file) to peptide length (Residues)
gene_to_peptide_length_map = dict(zip(peptide_stats_df["Systematic_ID"], peptide_stats_df["Residues"]))

print(f"Loaded peptide statistics for {len(gene_to_peptide_length_map)} proteins.")
print("Sample peptide lengths (Systematic ID: Length):")
for i, (gene_id, length) in enumerate(gene_to_peptide_length_map.items()):
    if i < 5:
        print(f"  {gene_id}: {length}")
    else:
        break

Loaded peptide statistics for 5134 proteins.
Sample peptide lengths (Systematic ID: Length):
  SPAC1002.01: 162
  SPAC1002.02: 229
  SPAC1002.03c: 923
  SPAC1002.04c: 199
  SPAC1002.05c: 715


## 5. Identify Gene Categories

From the parsed GFF data, we extract unique `Systematic ID`s for different categories of genes/transcripts, such as protein-coding genes (mRNAs), tRNAs, rRNAs, etc. This segmentation is useful for focusing analyses on specific types of genomic features.

In [113]:
# Get Systematic IDs associated with mRNA features, these are our coding genes
coding_gene_ids = gff_df[gff_df["Feature"] == 'mRNA']["Systematic ID"].unique().tolist()
trna_gene_ids = gff_df[gff_df["Feature"] == 'tRNA']["Systematic ID"].unique().tolist()
rrna_gene_ids = gff_df[gff_df["Feature"] == 'rRNA']["Systematic ID"].unique().tolist()
snorna_gene_ids = gff_df[gff_df["Feature"] == 'snoRNA']["Systematic ID"].unique().tolist()
snrna_gene_ids = gff_df[gff_df["Feature"] == 'snRNA']["Systematic ID"].unique().tolist()
lncrna_gene_ids = gff_df[gff_df["Feature"] == 'lncRNA']["Systematic ID"].unique().tolist()

print(f"Number of unique Systematic IDs for protein-coding genes (mRNA-associated): {len(coding_gene_ids)}")
print(f"Number of unique Systematic IDs for tRNA genes: {len(trna_gene_ids)}")
print(f"Number of unique Systematic IDs for rRNA genes: {len(rrna_gene_ids)}")
print(f"Number of unique Systematic IDs for snoRNA genes: {len(snorna_gene_ids)}")
print(f"Number of unique Systematic IDs for snRNA genes: {len(snrna_gene_ids)}")
print(f"Number of unique Systematic IDs for lncRNA genes: {len(lncrna_gene_ids)}")

# Example: Display feature counts for coding genes to verify
if coding_gene_ids:
    print(f"\nFeature counts for features associated with identified coding Systematic IDs:")
    coding_related_features = gff_df[gff_df["Systematic ID"].isin(coding_gene_ids)]
    display(coding_related_features["Feature"].value_counts())
else:
    print("No coding gene IDs found.")

Number of unique Systematic IDs for protein-coding genes (mRNA-associated): 5134
Number of unique Systematic IDs for tRNA genes: 196
Number of unique Systematic IDs for rRNA genes: 49
Number of unique Systematic IDs for snoRNA genes: 66
Number of unique Systematic IDs for snRNA genes: 7
Number of unique Systematic IDs for lncRNA genes: 7164

Feature counts for features associated with identified coding Systematic IDs:


Feature
CDS                10249
intron              5244
mRNA                5145
gene                5134
three_prime_UTR     4796
five_prime_UTR      4633
Name: count, dtype: int64

## 6. Process Coding Genes: CDS, Introns, and Primary Transcripts

This section focuses on processing protein-coding genes. We aim to:
1. Define a function to calculate the accumulated length of CDS (Coding DNA Sequence) segments for each transcript. This helps in understanding transcript structure, especially with alternative splicing.
2. Define a function to convert GFF features (specifically CDS and introns for coding genes) into BED format. This function also identifies the "primary transcript" using the peptide length information loaded earlier.

### 6.1. Calculate Accumulated CDS Bases

For each transcript, this function iterates through its features (CDS and introns) sorted by genomic position. It calculates the cumulative length of CDS features encountered up to that point *in the direction of transcription*. This information can be useful for mapping positions from a concatenated CDS sequence back to genomic coordinates or vice-versa.

In [114]:
def calculate_accumulated_cds_bases(transcript_features_df):
    """Calculates accumulated CDS bases for features within a transcript.
    
    Sorts features by start position. For negative strand genes, reverses the sorted order 
    to process features in the direction of transcription (3' to 5' genomically).
    The 'Accumulated_CDS_bases' for a feature is the sum of lengths of CDS segments 
    *before* it in the transcriptional order.
    
    Args:
        transcript_features_df (pd.DataFrame): DataFrame of features (e.g., CDS, intron) for a single transcript.
                                                Must contain 'Start', 'Strand', 'Feature', 'Length' columns.
    Returns:
        pd.DataFrame: The input DataFrame with an added 'Accumulated_CDS_bases' column.
    """
    if 'Length' not in transcript_features_df.columns or not pd.api.types.is_numeric_dtype(transcript_features_df['Length']):
        raise ValueError("DataFrame must contain a numeric 'Length' column.")
        
    # Sort by start position to process features in genomic order initially
    # Make a copy to avoid SettingWithCopyWarning
    sorted_df = transcript_features_df.sort_values(["Start"]).copy()
    
    current_cds_accumulation = 0
    strand = sorted_df["Strand"].iloc[0] # Assuming all features in the group have the same strand
    
    # Determine the order of iteration based on strand
    if strand == "+":
        iteration_indices = sorted_df.index
    elif strand == "-":
        iteration_indices = sorted_df.index[::-1] # Iterate from genomically last to first for '-' strand
    else:
        raise ValueError(f"Unknown strand: {strand} for transcript {sorted_df['Transcript'].iloc[0]}")

    sorted_df["Accumulated_CDS_bases"] = np.nan 

    for idx in iteration_indices:
        # The 'Accumulated_CDS_bases' for the current feature is the sum of CDS lengths *before* it.
        sorted_df.loc[idx, "Accumulated_CDS_bases"] = current_cds_accumulation
        if sorted_df.loc[idx, "Feature"] == "CDS":
            current_cds_accumulation += sorted_df.loc[idx, "Length"]
            
    return sorted_df

### 6.2. Convert GFF Features to BED Format for Coding Genes

The `gff_features_to_bed` function processes features of a single transcript (specifically CDS and introns for coding genes) and converts them into a BED-like format. Key steps include:
1. Adjusting coordinates: GFF is 1-based (start and end are inclusive), while BED is 0-based for start and 1-based for end (exclusive end). This function uses 0-based start, inclusive end as per common BED tool usage, but length calculation should be `End - Start`. For pybedtools, 0-based start, exclusive end is standard. We make `Start` 0-based.
2. Calculating feature lengths (`End - Start`).
3. Calling `calculate_accumulated_cds_bases` for CDS accumulation information.
4. Identifying the "primary transcript" by comparing the total CDS length to the expected protein length from `gene_to_peptide_length_map`. The protein length is typically (Total CDS length / 3) - 1 (if the stop codon is included in the CDS annotation and removed during translation). This logic should be verified against the specifics of the GFF annotation source.
5. Filtering features to be within the boundaries defined by the outermost CDS segments of the transcript.
6. Standardizing column names and order for the output BED-like DataFrame.

In [115]:
def gff_features_to_bed(transcript_features_group_df, gene_type_label, peptide_length_map):
    """Converts GFF features for a transcript (coding genes) to a BED-like format.

    Args:
        transcript_features_group_df (pd.DataFrame): DF of features (CDS, intron) for a single transcript group (grouped by Systematic ID and Transcript).
        gene_type_label (str): Label for the type of gene (e.g., "Coding gene").
        peptide_length_map (dict): Dictionary mapping Systematic ID to expected primary peptide length.

    Returns:
        pd.DataFrame or None: BED-formatted DataFrame for the transcript, or None if processing fails (e.g., no CDS found).
    """
    bed_df = transcript_features_group_df.copy()

    # Convert GFF 1-based start to BED 0-based start
    bed_df["Start"] = bed_df["Start"] - 1
    bed_df["Length"] = bed_df["End"] - bed_df["Start"] # Recalculate length after 0-basing start

    bed_columns_std = ["Chr", "Start", "End", "Transcript", "Length", "Strand"]
    other_info_cols = ["Feature", "Systematic ID", "Type"]

    current_gene_id = bed_df["Systematic ID"].iloc[0]
    current_transcript_id = bed_df["Transcript"].iloc[0]

    if gene_type_label == "Coding gene":
        boundary_feature_type = "CDS"
        cds_segments = bed_df[bed_df["Feature"] == boundary_feature_type]
        
        if cds_segments.empty:
            print(f"Warning: No CDS features found for transcript {current_transcript_id} of gene {current_gene_id}. Skipping.")
            return None 
            
        total_cds_length = cds_segments["Length"].sum()
        bed_df = calculate_accumulated_cds_bases(bed_df) # Adds 'Accumulated_CDS_bases'

        expected_peptide_len = peptide_length_map.get(current_gene_id, -1) 
        # PomBase CDS includes stop codon. Peptide length is (CDS length / 3) - 1.
        if expected_peptide_len != -1 and total_cds_length > 0 and (total_cds_length % 3 == 0) and \
           (int(total_cds_length / 3) - 1 == expected_peptide_len):
            bed_df["Primary_transcript_flag"] = "Yes"
        elif expected_peptide_len != -1 and total_cds_length > 0 and (total_cds_length % 3 != 0) and \
           (int(total_cds_length // 3) - 1 == expected_peptide_len):
            bed_df["Primary_transcript_flag"] = "Yes"
            print(f"Gene:{current_gene_id} Transcript:{current_transcript_id}: Length cannot be divied by 3 with 0 left!")
        else:
            bed_df["Primary_transcript_flag"] = "No"
        other_info_cols.extend(["Primary_transcript_flag", "Accumulated_CDS_bases"])
    elif gene_type_label == "Non-coding gene": 
        # For general non-coding RNAs ignoring the primary transcript flag
        boundary_feature_type = bed_df["Feature"].iloc[0]
    else:
        boundary_feature_type = "exon" # For non-coding RNA, exons define transcript extent
        # bed_df["Primary_transcript_flag"] = "N/A" 
        # other_info_cols.extend(["Primary_transcript_flag"])

    # Determine transcript boundaries based on the outermost specified features (CDS for coding, exon for others)
    boundary_defining_features = bed_df[bed_df["Feature"] == boundary_feature_type]
    if boundary_defining_features.empty:
        print(f"Warning: No '{boundary_feature_type}' features to define boundaries for {current_transcript_id}. Using all features for this transcript.")
        min_coord_start = bed_df["Start"].min()
        max_coord_end = bed_df["End"].max()
    else:
        min_coord_start = boundary_defining_features["Start"].min()
        max_coord_end = boundary_defining_features["End"].max()

    # Filter features to be within these boundaries
    filtered_bed_df = bed_df[(bed_df["Start"] >= min_coord_start) & (bed_df["End"] <= max_coord_end)].copy()

    if filtered_bed_df.empty:
        print(f"Warning: No features remained after boundary filtering for transcript {current_transcript_id}. Skipping.")
        return None
        
    filtered_bed_df.insert(3, "Type", gene_type_label) # Insert 'Type' column (e.g., "Coding gene")
    
    final_columns = bed_columns_std + other_info_cols
    
    # Ensure all expected columns are present, fill with NaN if not, and order them
    for col in final_columns:
        if col not in filtered_bed_df.columns:
            filtered_bed_df[col] = np.nan
            
    # Rename Chr to #Chr for bedtools convention and sort
    output_df = filtered_bed_df[final_columns].rename(columns={"Chr": "#Chr"}) \
                                 .sort_values(["#Chr"] + bed_columns_std[1:]) 
    
    return output_df

### 6.3. Apply Processing to Coding Genes

Now we filter the main GFF DataFrame for features relevant to coding genes (CDS and introns associated with `coding_gene_ids`). Then, we group these features by `Systematic ID` and `Transcript` and apply the `gff_features_to_bed` function to each group.

In [116]:
# Filter for CDS and intron features associated with the identified coding gene IDs
coding_gene_related_features_df = gff_df[
    gff_df["Systematic ID"].isin(coding_gene_ids) & 
    gff_df["Feature"].isin(['CDS', 'intron'])
].copy()

print(f"Processing {coding_gene_related_features_df['Transcript'].nunique()} unique transcript IDs from coding genes.")

processed_coding_beds_list = []
if not coding_gene_related_features_df.empty:
    grouped_transcripts = coding_gene_related_features_df.groupby(["Systematic ID", "Transcript"])
    for name, group in grouped_transcripts:
        try:
            # Ensure group has all necessary columns from gff_features_to_bed before passing
            # 'Start', 'End', 'Strand', 'Feature', 'Systematic ID', 'Transcript' are expected from gff_df structure
            processed_bed_for_transcript = gff_features_to_bed(group, "Coding gene", gene_to_peptide_length_map)
            if processed_bed_for_transcript is not None and not processed_bed_for_transcript.empty:
                processed_coding_beds_list.append(processed_bed_for_transcript)
        except Exception as e:
            print(f"Error processing group {name} (Systematic ID: {name[0]}, Transcript: {name[1]}): {e}")

if processed_coding_beds_list:
    all_coding_features_bed_df = pd.concat(processed_coding_beds_list).reset_index(drop=True)
    print("\nProcessed Coding Gene Features (CDS/Introns) in BED-like format:")
    display(all_coding_features_bed_df.head())
    print(f"Shape of all_coding_features_bed_df: {all_coding_features_bed_df.shape}")
    print(f"Number of unique Transcripts in all_coding_features_bed_df: {all_coding_features_bed_df['Transcript'].nunique()}")
    # Display distribution of Primary_transcript_flag for unique transcripts
    if 'Primary_transcript_flag' in all_coding_features_bed_df.columns:
        primary_flag_dist = all_coding_features_bed_df.drop_duplicates(subset=['Transcript'])['Primary_transcript_flag'].value_counts()
        print(f"\nPrimary Transcript Flag distribution (among unique transcripts):\n{primary_flag_dist}")
else:
    print("No coding gene BED features were successfully processed or generated.")
    all_coding_features_bed_df = pd.DataFrame() # Ensure it exists as an empty df for subsequent steps

Processing 5145 unique transcript IDs from coding genes.
Gene:SPAC212.11 Transcript:SPAC212.11.1: Length cannot be divied by 3 with 0 left!
Gene:SPAC977.01 Transcript:SPAC977.01.1: Length cannot be divied by 3 with 0 left!

Processed Coding Gene Features (CDS/Introns) in BED-like format:


,#Chr,Start,End,Transcript,Length,Strand,Feature,Systematic ID,Type,Primary_transcript_flag,Accumulated_CDS_bases
0,I,1798346,1798835,SPAC1002.01.1,489,+,CDS,SPAC1002.01,Coding gene,Yes,0.0
1,I,1799127,1799817,SPAC1002.02.1,690,+,CDS,SPAC1002.02,Coding gene,Yes,0.0
2,I,1800211,1802983,SPAC1002.03c.1,2772,-,CDS,SPAC1002.03c,Coding gene,Yes,0.0
3,I,1803772,1804372,SPAC1002.04c.1,600,-,CDS,SPAC1002.04c,Coding gene,Yes,0.0
4,I,1804649,1806797,SPAC1002.05c.1,2148,-,CDS,SPAC1002.05c,Coding gene,Yes,0.0


Shape of all_coding_features_bed_df: (15333, 11)
Number of unique Transcripts in all_coding_features_bed_df: 5145

Primary Transcript Flag distribution (among unique transcripts):
Primary_transcript_flag
Yes    5136
No        9
Name: count, dtype: int64


## 7. Refine Primary Transcript Selection and Create Primary Transcript BED

Some genes might have multiple annotated transcripts that yield the same protein (identical CDS length and sequence) but differ in their UTRs. The `Primary_transcript_flag` helped identify transcripts matching protein length. The original notebook had a step to prefer transcripts ending with `.1` if multiple such primary candidates existed for a gene. We will implement a similar disambiguation if needed, then filter for primary transcripts.

In [117]:
primary_transcripts_bed_df = pd.DataFrame() # Initialize an empty DataFrame

if not all_coding_features_bed_df.empty and 'Primary_transcript_flag' in all_coding_features_bed_df.columns:
    # Get unique transcripts that were flagged as 'Yes'
    candidate_primary_transcripts = all_coding_features_bed_df[
        all_coding_features_bed_df['Primary_transcript_flag'] == 'Yes'
    ][[ 'Systematic ID', 'Transcript']].drop_duplicates()
    
    # Count how many such primary candidates exist per gene
    primary_candidates_per_gene = candidate_primary_transcripts.groupby('Systematic ID')['Transcript'].count()
    genes_with_multiple_primary = primary_candidates_per_gene[primary_candidates_per_gene > 1].index.tolist()
    
    final_primary_transcript_ids = []
    for gene_id, group in candidate_primary_transcripts.groupby('Systematic ID'):
        if gene_id in genes_with_multiple_primary:
            # Prefer .1 transcripts if multiple primary candidates exist
            dot_one_transcripts = [tid for tid in group['Transcript'] if tid.endswith('.1')]
            if dot_one_transcripts:
                final_primary_transcript_ids.append(dot_one_transcripts[0]) # Take the first .1 found
            else:
                final_primary_transcript_ids.append(group['Transcript'].iloc[0]) # Fallback to the first one listed
        else:
            final_primary_transcript_ids.append(group['Transcript'].iloc[0]) # Only one candidate
            
    # Filter the main bed df for these final primary transcript IDs
    primary_transcripts_bed_df = all_coding_features_bed_df[
        all_coding_features_bed_df['Transcript'].isin(final_primary_transcript_ids)
    ].copy()
    
    # Ensure correct chromosome order and sort
    if not primary_transcripts_bed_df.empty:
        # Ensure '#Chr' is categorical with the correct order
        # Use chr_order defined during initial GFF parsing
        chr_order = ["chr_II_telomeric_gap", "I", "II", "III", "mating_type_region", "mitochondrial"]
        primary_transcripts_bed_df["#Chr"] = pd.Categorical(primary_transcripts_bed_df["#Chr"], categories=chr_order, ordered=True)
        primary_transcripts_bed_df = primary_transcripts_bed_df.sort_values(["#Chr", "Start", "End"])
        print("\nSelected Primary Transcripts Features (BED-like format):")
        display(primary_transcripts_bed_df.head())
        print(f"Shape of primary_transcripts_bed_df: {primary_transcripts_bed_df.shape}")
        print(f"Number of unique primary Transcripts selected: {primary_transcripts_bed_df['Transcript'].nunique()}")
    else:
        print("No primary transcripts selected after disambiguation or none met criteria.")
else:
    print("Skipping primary transcript refinement as initial coding features BED is empty or lacks primary flag.")


Selected Primary Transcripts Features (BED-like format):


,#Chr,Start,End,Transcript,Length,Strand,Feature,Systematic ID,Type,Primary_transcript_flag,Accumulated_CDS_bases
11051,chr_II_telomeric_gap,1478,3197,SPBC460.01c.1,1719,-,CDS,SPBC460.01c,Coding gene,Yes,0.0
11052,chr_II_telomeric_gap,8855,9365,SPBC460.02c.1,510,-,CDS,SPBC460.02c,Coding gene,Yes,153.0
11053,chr_II_telomeric_gap,9365,9650,SPBC460.02c.1,285,-,intron,SPBC460.02c,Coding gene,Yes,153.0
11054,chr_II_telomeric_gap,9650,9803,SPBC460.02c.1,153,-,CDS,SPBC460.02c,Coding gene,Yes,0.0
11055,chr_II_telomeric_gap,11640,13344,SPBC460.03.1,1704,+,CDS,SPBC460.03,Coding gene,Yes,0.0


Shape of primary_transcripts_bed_df: (15254, 11)
Number of unique primary Transcripts selected: 5134


## 8. Identify Intergenic Regions

Intergenic regions are genomic segments that do not overlap with any annotated genes (or in this case, our defined primary transcripts). We use `pybedtools` to find these regions. The `complement` tool in BedTools takes a set of genomic intervals and returns their inverse within the bounds of the genome (defined by the FASTA index `.fai` file).

### 8.1. Prepare Primary Transcripts for BedTools

To use `pybedtools.complement`, we first need to define the overall span of each primary transcript, not its individual CDS/intron features. We'll group the `primary_transcripts_bed_df` by `Transcript` and find the min `Start` and max `End` for each to represent the transcript's full extent.

In [118]:
intergenic_regions_df = pd.DataFrame() # Initialize an empty DataFrame

if not primary_transcripts_bed_df.empty:

    # Create a BedTool object from the transcript spans
    try:
        primary_transcripts_bedtool = BedTool.from_dataframe(primary_transcripts_bed_df)
        
        # Find complementary regions (intergenic)
        # The 'g' argument requires a genome file (chromosome sizes), typically from a .fai file or a dictionary.
        intergenic_bedtool = primary_transcripts_bedtool.complement(g=fai_file_path)
        
        # Convert back to DataFrame
        intergenic_regions_df = intergenic_bedtool.to_dataframe(disable_auto_names=True, header=None)
        # Assign standard BED column names if to_dataframe doesn't infer them well or if header=None
        if intergenic_regions_df.shape[1] >= 3: # Expect at least chrom, start, end
            col_names = ['#Chr', 'Start', 'End'] + [f'col_{i+4}' for i in range(intergenic_regions_df.shape[1] - 3)]
            intergenic_regions_df.columns = col_names[:intergenic_regions_df.shape[1]]
        
        print("\nIdentified Intergenic Regions:")
        display(intergenic_regions_df.head())
        print(f"Shape of intergenic_regions_df: {intergenic_regions_df.shape}")
    except Exception as e:
        print(f"Error during BedTools operations: {e}")
        intergenic_regions_df = pd.DataFrame() # Ensure it's an empty df on error
else:
    print("Primary transcripts BED DataFrame is empty. Skipping intergenic region identification.")


Identified Intergenic Regions:


,#Chr,Start,End
0,chr_II_telomeric_gap,0,1478
1,chr_II_telomeric_gap,3197,8855
2,chr_II_telomeric_gap,9803,11640
3,chr_II_telomeric_gap,13344,14072
4,chr_II_telomeric_gap,15248,16469


Shape of intergenic_regions_df: (5106, 3)


## 9. Annotate Intergenic Regions

The identified intergenic regions are currently just coordinates. To make them more informative, we'll annotate them with information about their flanking (neighboring) primary transcripts. This involves finding which transcript ends at the start of an intergenic region and which transcript starts at the end of it.

In [119]:
def annotate_intergenic_region_flanks(intergenic_row, primary_transcripts_bed_df):
    """Annotates an intergenic region with its left and right flanking transcripts.
    
    Args:
        intergenic_row (pd.Series): A row from the intergenic regions DataFrame (must have '#Chr', 'Start', 'End').
        primary_transcripts_bed_df
    Returns:
        pd.Series: Series with flanking information (Left_Transcript, Right_Transcript, etc.).
    """
    chrom = intergenic_row["#Chr"]
    intergenic_start = intergenic_row["Start"]
    intergenic_end = intergenic_row["End"]
    
    # Find left flank (transcript whose End matches intergenic_start)
    left_flank_candidates = primary_transcripts_bed_df[
        (primary_transcripts_bed_df["#Chr"] == chrom) & 
        (primary_transcripts_bed_df["End"] == intergenic_start)
    ]
    
    # Find right flank (transcript whose Start matches intergenic_end)
    right_flank_candidates = primary_transcripts_bed_df[
        (primary_transcripts_bed_df["#Chr"] == chrom) & 
        (primary_transcripts_bed_df["Start"] == intergenic_end)
    ]
    
    flank_info = {
        'Left_Transcript': np.nan, 'Left_Systematic ID': np.nan, 'Left_Strand': np.nan,
        'Right_Transcript': np.nan, 'Right_Systematic ID': np.nan, 'Right_Strand': np.nan,
        'Orientation': np.nan
    }
    
    if not left_flank_candidates.empty:
        left_flank = left_flank_candidates.iloc[0] # Take the first if multiple (rare)
        flank_info['Left_Transcript'] = left_flank['Transcript']
        flank_info['Left_Systematic ID'] = left_flank['Systematic ID']
        flank_info['Left_Strand'] = left_flank['Strand']
    else:
        flank_info['Left_Transcript'] = 'Boundary'
        flank_info['Left_Systematic ID'] = 'Boundary'
        flank_info['Left_Strand'] = 'Boundary'
        
    if not right_flank_candidates.empty:
        right_flank = right_flank_candidates.iloc[0]
        flank_info['Right_Transcript'] = right_flank['Transcript']
        flank_info['Right_Systematic ID'] = right_flank['Systematic ID']
        flank_info['Right_Strand'] = right_flank['Strand']
    else:
        flank_info['Right_Transcript'] = 'Boundary'
        flank_info['Right_Systematic ID'] = 'Boundary'
        flank_info['Right_Strand'] = 'Boundary'

    # Determine orientation (convergent, divergent, tandem)
    if flank_info['Left_Strand'] == '+' and flank_info['Right_Strand'] == '-':
        flank_info['Orientation'] = 'Convergent' # --> <--
    elif flank_info['Left_Strand'] == '-' and flank_info['Right_Strand'] == '+':
        flank_info['Orientation'] = 'Divergent'  # <-- -->
    elif flank_info['Left_Strand'] == '+' and flank_info['Right_Strand'] == '+':
        flank_info['Orientation'] = 'Tandem_Plus' # --> -->
    elif flank_info['Left_Strand'] == '-' and flank_info['Right_Strand'] == '-':
        flank_info['Orientation'] = 'Tandem_Minus' # <-- <--
    elif 'Boundary' in [str(flank_info['Left_Strand']), str(flank_info['Right_Strand'])]:
        flank_info['Orientation'] = 'Boundary_Adjacent'
    
    flank = {
        "Transcript": flank_info['Left_Transcript'] + "|" + flank_info['Right_Transcript'],
        "Systematic ID": flank_info['Left_Systematic ID'] + "|" + flank_info['Right_Systematic ID'],
        "Strand": flank_info['Left_Strand'] + "|" + flank_info['Right_Strand'],
        "Feature": flank_info['Orientation'] + "_Region"
    }
        
    return pd.Series(flank)

if not intergenic_regions_df.empty and not primary_transcripts_bed_df.empty:
    intergenic_regions_df[["Transcript", "Systematic ID", "Strand", "Feature"]] = intergenic_regions_df.apply(
        annotate_intergenic_region_flanks,
        primary_transcripts_bed_df=primary_transcripts_bed_df, 
        axis=1)
    
    # Add Length, Feature type, etc.
    intergenic_regions_df["Length"] = intergenic_regions_df["End"] - intergenic_regions_df["Start"]
    intergenic_regions_df["Type"] = "Intergenic region"
    intergenic_regions_df = intergenic_regions_df[["#Chr", "Start", "End", "Transcript", "Length", "Strand", "Feature", "Systematic ID", "Type"]].copy()
    print("\nAnnotated Intergenic Regions:")
    display(intergenic_regions_df.head())
    print(f"Shape of annotated_intergenic_df: {intergenic_regions_df.shape}")
    if 'Orientation' in intergenic_regions_df.columns:
        print("\nOrientation of intergenic regions:")
        display(intergenic_regions_df['Orientation'].value_counts(dropna=False))
else:
    print("Intergenic regions DataFrame or primary transcripts data is empty. Skipping annotation.")


Annotated Intergenic Regions:


,#Chr,Start,End,Transcript,Length,Strand,Feature,Systematic ID,Type
0,chr_II_telomeric_gap,0,1478,Boundary|SPBC460.01c.1,1478,Boundary|-,Boundary_Adjacent_Region,Boundary|SPBC460.01c,Intergenic region
1,chr_II_telomeric_gap,3197,8855,SPBC460.01c.1|SPBC460.02c.1,5658,-|-,Tandem_Minus_Region,SPBC460.01c|SPBC460.02c,Intergenic region
2,chr_II_telomeric_gap,9803,11640,SPBC460.02c.1|SPBC460.03.1,1837,-|+,Divergent_Region,SPBC460.02c|SPBC460.03,Intergenic region
3,chr_II_telomeric_gap,13344,14072,SPBC460.03.1|SPBC460.04c.1,728,+|-,Convergent_Region,SPBC460.03|SPBC460.04c,Intergenic region
4,chr_II_telomeric_gap,15248,16469,SPBC460.04c.1|SPBC460.05.1,1221,-|+,Divergent_Region,SPBC460.04c|SPBC460.05,Intergenic region


Shape of annotated_intergenic_df: (5106, 9)


## 10. Process non-coding genes: tRNA, rRNA, snoRNA, snRNA, lncRNA.

In [120]:
# Define the list of non-coding RNA feature types we are interested in
# These are based on common non-coding RNA categories and what's available in the GFF
# (refer to gff_df_cleaned['Feature'].value_counts() output from earlier cells if needed)
non_coding_rna_features = [
    'tRNA',          # Transfer RNA
    'rRNA',          # Ribosomal RNA
    'snoRNA',   # Small nucleolar RNA gene (often the 'gene' entry is what we want for annotation)
    'snRNA',         # Small nuclear RNA
    'lncRNA'    # Long non-coding RNA gene
    # Note: Depending on GFF annotation practices, you might need to look for transcript-level features
    # like 'snoRNA', 'lnc_RNA' if 'snoRNA_gene' or 'lncRNA_gene' are not the desired level of detail.
    # For this example, we'll use the gene-level entries for snoRNA and lncRNA if they are more common.
]

# Filter the GFF DataFrame for these non-coding RNA features
non_coding_rna_df = gff_df[gff_df['Feature'].isin(non_coding_rna_features)].copy().sort_values(["Feature", "Chr", "Start", "End", "Systematic ID", "Transcript"])

# process the non-coding RNA features to bed
non_coding_rna_bed_df = non_coding_rna_df.groupby("Feature").apply(gff_features_to_bed, "Non-coding gene", gene_to_peptide_length_map).reset_index(drop=True)

# display the information of the non-coding RNA features
print(f"Shape of non_coding_rna_bed_df: {non_coding_rna_bed_df.shape}")
print("Before processing:")
display(non_coding_rna_df.head())
print("After processing:")
display(non_coding_rna_bed_df.head())
# display the counts of the non-coding RNA features
print(f"Counts of non-coding RNA feature types:")
display(non_coding_rna_bed_df['Feature'].value_counts())
# display each feature type
for feature in non_coding_rna_features:
    print(f"Feature type: {feature}")
    display(non_coding_rna_bed_df[non_coding_rna_bed_df['Feature'] == feature].head())
    print(f"Shape of non_coding_rna_bed_df for feature type {feature}: {non_coding_rna_bed_df[non_coding_rna_bed_df['Feature'] == feature].shape}")

Shape of non_coding_rna_bed_df: (7484, 9)
Before processing:


,Chr,Source,Feature,Start,End,Score,Strand,Frame,Attribute,Systematic ID,Transcript
38345,I,PomBase,lncRNA,346,1164,.,+,.,ID=SPNCRNA.2000.1;Parent=SPNCRNA.2000,SPNCRNA.2000,SPNCRNA.2000.1
38348,I,PomBase,lncRNA,2118,2347,.,+,.,ID=SPNCRNA.2001.1;Parent=SPNCRNA.2001,SPNCRNA.2001,SPNCRNA.2001.1
38351,I,PomBase,lncRNA,2448,2760,.,+,.,ID=SPNCRNA.2002.1;Parent=SPNCRNA.2002,SPNCRNA.2002,SPNCRNA.2002.1
38354,I,PomBase,lncRNA,4057,4311,.,+,.,ID=SPNCRNA.2003.1;Parent=SPNCRNA.2003,SPNCRNA.2003,SPNCRNA.2003.1
38357,I,PomBase,lncRNA,4784,5129,.,+,.,ID=SPNCRNA.2004.1;Parent=SPNCRNA.2004,SPNCRNA.2004,SPNCRNA.2004.1


After processing:


,#Chr,Start,End,Transcript,Length,Strand,Feature,Systematic ID,Type
0,I,345,1164,SPNCRNA.2000.1,819,+,lncRNA,SPNCRNA.2000,Non-coding gene
1,I,2117,2347,SPNCRNA.2001.1,230,+,lncRNA,SPNCRNA.2001,Non-coding gene
2,I,2447,2760,SPNCRNA.2002.1,313,+,lncRNA,SPNCRNA.2002,Non-coding gene
3,I,4056,4311,SPNCRNA.2003.1,255,+,lncRNA,SPNCRNA.2003,Non-coding gene
4,I,4783,5129,SPNCRNA.2004.1,346,+,lncRNA,SPNCRNA.2004,Non-coding gene


Counts of non-coding RNA feature types:


...

Shape of non_coding_rna_bed_df for feature type snoRNA: (66, 9)
Feature type: snRNA


,#Chr,Start,End,Transcript,Length,Strand,Feature,Systematic ID,Type
7215,I,959369,959556,SPSNRNA.02.1,187,-,snRNA,SPSNRNA.02,Non-coding gene
7216,I,2562275,2562419,SPSNRNA.06.1,144,-,snRNA,SPSNRNA.06,Non-coding gene
7217,II,467488,467615,SPSNRNA.04.1,127,-,snRNA,SPSNRNA.04,Non-coding gene
7218,II,1308235,1308343,SPSNORNA.55.1,108,+,snRNA,SPSNORNA.55,Non-coding gene
7219,II,3020204,3020353,SPSNRNA.01.1,149,+,snRNA,SPSNRNA.01,Non-coding gene


Shape of non_coding_rna_bed_df for feature type snRNA: (7, 9)
Feature type: lncRNA


,#Chr,Start,End,Transcript,Length,Strand,Feature,Systematic ID,Type
0,I,345,1164,SPNCRNA.2000.1,819,+,lncRNA,SPNCRNA.2000,Non-coding gene
1,I,2117,2347,SPNCRNA.2001.1,230,+,lncRNA,SPNCRNA.2001,Non-coding gene
2,I,2447,2760,SPNCRNA.2002.1,313,+,lncRNA,SPNCRNA.2002,Non-coding gene
3,I,4056,4311,SPNCRNA.2003.1,255,+,lncRNA,SPNCRNA.2003,Non-coding gene
4,I,4783,5129,SPNCRNA.2004.1,346,+,lncRNA,SPNCRNA.2004,Non-coding gene


Shape of non_coding_rna_bed_df for feature type lncRNA: (7166, 9)


## 11. Add other information

### 11.1. Add gene name

In [121]:
gene_IDs_names_products = pd.read_csv(gene_IDs_names_products_path, sep="\t")
gene_IDs_names_products["gene_name"] = gene_IDs_names_products["gene_name"].fillna(gene_IDs_names_products["gene_systematic_id"])
ID2name = dict(zip(gene_IDs_names_products["gene_systematic_id"], gene_IDs_names_products["gene_name"]))

for bed in [primary_transcripts_bed_df, intergenic_regions_df, non_coding_rna_bed_df]:
    if not bed.empty:
        if bed["Systematic ID"].str.contains("|").any():
            bed["Name"] = bed["Systematic ID"].apply(lambda x: "|".join([ID2name.get(i, i) for i in x.split("|")]))
        else:
            bed["Name"] = bed["Systematic ID"].map(ID2name)


### 11.2. Add gene essentiality

In [122]:
FYPOviability_df = pd.read_csv(FYPOviability_path, sep="\t", header=None, names=["Systematic ID", "FYPOviability"])
FYPOviability = FYPOviability_df.set_index("Systematic ID")["FYPOviability"].to_dict()

Hayles_viability_df = pd.read_excel(Hayles_viability_path)
Hayles_viability_df["Updated_Systematic_ID"] = update_sysID(Hayles_viability_df["Systematic ID"], gene_IDs_names_products)
DeletionLibrary_essentiality = dict(zip(Hayles_viability_df["Updated_Systematic_ID"], Hayles_viability_df["Gene dispensability. This study"].str.strip()))

SPAC1556.06.1 is updated to SPAC1556.06
SPAC823.02 is updated to SPAC823.17
SPBC8E4.02c is not found in geneid2symbol or synonyms2ID


In [123]:
for bed in [primary_transcripts_bed_df, intergenic_regions_df, non_coding_rna_bed_df]:
    if not bed.empty:
        if bed["Systematic ID"].str.contains("|").any():
            bed["FYPOviability"] = bed["Systematic ID"].apply(lambda x: "|".join([FYPOviability.get(i, i) for i in x.split("|")]))
            bed["DeletionLibrary_essentiality"] = bed["Systematic ID"].apply(lambda x: "|".join([DeletionLibrary_essentiality.get(i, "Not_determined") for i in x.split("|")]))
        else:
            bed["FYPOviability"] = bed["Systematic ID"].map(FYPOviability)
            bed["DeletionLibrary_essentiality"] = bed["Systematic ID"].map(DeletionLibrary_essentiality)

### 11.3 Add parental region information

In [124]:
for bed in [primary_transcripts_bed_df, intergenic_regions_df, non_coding_rna_bed_df]:
    if not bed.empty:
        bed["ParentalRegion_start"] = bed.groupby("Systematic ID")["Start"].transform("min")
        bed["ParentalRegion_end"] = bed.groupby("Systematic ID")["End"].transform("max")
        bed["ParentalRegion_length"] = bed["ParentalRegion_end"] - bed["ParentalRegion_start"]        


### 11.4 Find overlapping regions

In [125]:
def find_overlapping_regions(feature):
    chr_a, chr_b = feature[0], feature[6]
    start_a, start_b = int(feature[1]), int(feature[7])
    end_a, end_b = int(feature[2]), int(feature[8])
    transcript_a, transcript_b = feature[3], feature[9]
    sysID_a, sysID_b = feature[4], feature[10]
    strand_a, strand_b = feature[5], feature[11]

    chrom = chr_a
    start = max(start_a, start_b)
    end = min(end_a, end_b)
    transcript = transcript_a
    sysID = sysID_a
    strand = strand_a
    score = ""
    if transcript_a != transcript_b:
        transcript = transcript_a + "," + transcript_b
        sysID = sysID_a + "," + sysID_b
        strand = strand_a + "," + strand_b
        score = "Overlapping genes"
        return pybedtools.create_interval_from_list(
            [chrom, str(start), str(end), transcript, strand, score, sysID]
        )
    else:
        return pybedtools.create_interval_from_list(
            [chrom, str(start), str(end), transcript, strand, score, sysID]
        )

In [126]:
primary_transcripts_bed = BedTool.from_dataframe(primary_transcripts_bed_df[["#Chr", "ParentalRegion_start", "ParentalRegion_end", "Transcript", "Systematic ID", "Strand"]].drop_duplicates())

# Find overlapping intervals
overlaps = primary_transcripts_bed.intersect(
    primary_transcripts_bed, wa=True, wb=True, header=True)

# Process overlaps
results = overlaps.each(find_overlapping_regions)

# drop duplicate lines
results = (
    results.saveas()
    .to_dataframe(names=["#Chr", "Start", "End", "Transcript", "Strand", "Feature", "Systematic ID"])
    .drop_duplicates(subset=["#Chr", "Start", "End"], keep="first")
)

# overlapped genes
overlapped_region_bed_df = results[results["Feature"] == "Overlapping genes"].copy()
overlapped_region_bed_df["Length"] = (
    overlapped_region_bed_df["End"] - overlapped_region_bed_df["Start"]
)
overlapped_region_bed_df["Type"] = "Coding gene"
overlapped_region_bed_df = overlapped_region_bed_df[["#Chr", "Start", "End", "Transcript", "Length", "Strand", "Feature", "Systematic ID", "Type"]].copy()

overlapped_region_bed_df["Name"] = overlapped_region_bed_df["Systematic ID"].apply(lambda x: ",".join([ID2name.get(i, i) for i in x.split(",")]))
overlapped_region_bed_df["FYPOviability"] = overlapped_region_bed_df["Systematic ID"].apply(lambda x: ",".join([FYPOviability.get(i, i) for i in x.split(",")]))
overlapped_region_bed_df["DeletionLibrary_essentiality"] = overlapped_region_bed_df["Systematic ID"].apply(lambda x: ",".join([DeletionLibrary_essentiality.get(i, "Not_determined") for i in x.split(",")]))
overlapped_region_bed_df



***** WARNING: File /tmp/pybedtools.lxa_4b1v.tmp has inconsistent naming convention for record:
I	0	5662	SPAC212.11.1	SPAC212.11	-

***** WARNING: File /tmp/pybedtools.lxa_4b1v.tmp has inconsistent naming convention for record:
I	0	5662	SPAC212.11.1	SPAC212.11	-



,#Chr,Start,End,Transcript,Length,Strand,Feature,Systematic ID,Type,Name,FYPOviability,DeletionLibrary_essentiality
283,I,698032,698068,"SPAC22F3.05c.1,SPAC22F3.04.1",36,"+,-",Overlapping genes,"SPAC22F3.05c,SPAC22F3.04",Coding gene,"alp41,mug62","inviable,viable","E,V"
363,I,905128,905132,"SPAC1687.02.1,SPAC1687.03c.1",4,"+,-",Overlapping genes,"SPAC1687.02,SPAC1687.03c",Coding gene,"rce1,rfc4","inviable,inviable","E,E"
1209,I,2828665,2828687,"SPAC3F10.06c.1,SPAC3F10.07c.1",22,"-,-",Overlapping genes,"SPAC3F10.06c,SPAC3F10.07c",Coding gene,"rit1,erf4","viable,viable","V,Not_determined"
1215,I,2833890,2833974,"SPAC3F10.09.1,SPAC3F10.10c.1",84,"+,-",Overlapping genes,"SPAC3F10.09,SPAC3F10.10c",Coding gene,"his6,map3","viable,viable","V,V"
1330,I,3106839,3106841,"SPAC589.08c.1,SPAC589.09.1",2,"-,+",Overlapping genes,"SPAC589.08c,SPAC589.09",Coding gene,"dam1,csr101","viable,viable","V,V"
1353,I,3152778,3152782,"SPAC3G9.13c.1,SPAC3G9.12.1",4,"+,-",Overlapping genes,"SPAC3G9.13c,SPAC3G9.12",Coding gene,"msw1,peg1","inviable,inviable","E,E"
1455,I,3392258,3392274,"SPAC959.11.1,SPAC959.10.1",16,"-,+",Overlapping genes,"SPAC959.11,SPAC959.10",Coding gene,"SPAC959.11,sen15","unknown,inviable","Not_determined,E"
1705,I,4010912,4011249,"SPAC27E2.04c.1,SPAC27E2.12.1",337,"-,+",Overlapping genes,"SPAC27E2.04c,SPAC27E2.12",Coding gene,"mug155,SPAC27E2.12","unknown,viable","Not_determined,Not_determined"
2172,I,5098186,5098212,"SPAC1250.07.1,SPAC1250.03.1",26,"-,-",Overlapping genes,"SPAC1250.07,SPAC1250.03",Coding gene,"sfc7,ubc14","condition-dependent,viable","Not_determined,V"
2184,I,5120872,5120876,"SPAC29A4.14c.1,SPAC29A4.13.1",4,"+,-",Overlapping genes,"SPAC29A4.14c,SPAC29A4.13",Coding gene,"pex3,ure6","viable,viable","V,V"


## 12. (Optional) Save Processed Data

At this stage, you might want to save the processed DataFrames (e.g., `primary_transcripts_bed_df`, `annotated_intergenic_df`) to files (CSV, BED, etc.) for use in other analyses or tools.

In [129]:
# Saving the primary transcripts features (CDS/introns) BED
if not primary_transcripts_bed_df.empty:
    primary_transcripts_bed_df.drop(columns=["Primary_transcript_flag"]).to_csv(coding_gene_primary_transcripts_bed_path, sep="\t", index=False)
    print("Saved coding_genes_primary_transcripts_features.bed")

# Saving the annotated intergenic regions
if not intergenic_regions_df.empty:
    # For BED format, select minimal columns: #Chr, Start, End, Name (e.g., intergenic ID or flanking info), Score, Strand
    # Construct a suitable name field, e.g., by combining flanking gene IDs
    # annotated_intergenic_df['name_for_bed'] = annotated_intergenic_df['Left_Systematic ID'] + '_' + annotated_intergenic_df['Right_Systematic ID'] 
    # intergenic_to_save = annotated_intergenic_df[['#Chr', 'Start', 'End', 'name_for_bed']] # Add score and strand if meaningful
    intergenic_regions_df.to_csv(intergenic_regions_bed_path, sep="\t", index=False)
    print("Saved intergenic_regions.bed")

# Saving the non-coding RNA features
if not non_coding_rna_bed_df.empty:
    non_coding_rna_bed_df.to_csv(non_coding_rna_bed_path, sep="\t", index=False)
    print("Saved non_coding_rna_features.bed")

genome_intervals_bed_df = pd.concat([primary_transcripts_bed_df.drop(columns=["Primary_transcript_flag"]), intergenic_regions_df])
genome_intervals_bed_df.to_csv(Genome_intervals_bed_path, sep="\t", index=False)
print("Saved genome_intervals.bed")

if not overlapped_region_bed_df.empty:
    overlapped_region_bed_df.to_csv(overlapped_region_bed_path, sep="\t", index=False)
    print("Saved overlapped_region.bed")


Saved coding_genes_primary_transcripts_features.bed
Saved intergenic_regions.bed
Saved non_coding_rna_features.bed
Saved genome_intervals.bed
Saved overlapped_region.bed


## 11. Conclusion

This notebook has demonstrated a workflow for parsing a GFF3 file, identifying and processing coding genes to determine primary transcripts, and subsequently finding and annotating intergenic regions. The resulting DataFrames provide a structured representation of these key genomic features, suitable for various downstream bioinformatics analyses.